In [1]:
import pandas as pd
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from tavily import TavilyClient
from langchain_experimental.agents import create_csv_agent
from langchain_groq import ChatGroq
from sqlalchemy import create_engine
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.utilities import SQLDatabase

In [2]:
model = ChatGroq(
            model="llama3-70b-8192",groq_api_key="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", temperature= 0
        )

In [3]:
def analyst(input=""):
    model = ChatGroq(
            model="llama3-70b-8192",groq_api_key="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", temperature= 0
        )
    
    csv_path ="people.csv"
    agent = create_csv_agent(model, csv_path ,allow_dangerous_code=True)
    response = agent.run(input)
    return(response)
analyst_tool=Tool(
    name="Data_analyst",
    func=analyst,
    description=" use this tool to analyze a CSV dataset using a pre-trained large language mode.It interacts with the model to answer questions about the data and if need you can also use python language for task completion"
)

In [4]:

def sql(input=""):
    model = ChatGroq(
                model="llama3-70b-8192",groq_api_key="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", temperature= 0
            )
    engine = create_engine("sqlite:///my_database.db")
    db = SQLDatabase(engine=engine)
    agent_executor = create_sql_agent(model, db=db, verbose=True)
    return(agent_executor.invoke({"input": {input}}))
SQL_Tool=Tool(
    name="Sql_tool",
    func=sql,
    description="You are a SQL tool and you need to answer to the inputs given to you by generating SQL queries and return the output"
)
    

In [5]:
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer
def scrape(input=""):
    urls = [input]
    loader = AsyncHtmlLoader(urls)
    docs = loader.load()
    html2text = Html2TextTransformer()
    docs_transformed = html2text.transform_documents(docs)
    return(docs_transformed[0].page_content[0:500])
web_scraper_tool=Tool(
    name="Web_scrapper",
    func=scrape,
    description="Scrape the Given url provided by the user and give a short description about it"
)
    

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
def search(input=""):
    tavily = TavilyClient(api_key="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")
    query = input
    response = tavily.search(query)
    if response["results"][0]:
        result=response["results"][0]
        return(result.get("content"))
search_tool=Tool(
    name="Tavily_search_engine",
    func=search,
    description="You are a search tool and you need to give answers to the query"
)

In [7]:
tools=[analyst_tool,search_tool,web_scraper_tool,SQL_Tool]

In [8]:
memory=ConversationBufferWindowMemory(memory_key='chat_history',k=5,return_messages=True)

In [9]:
conversational_agent =initialize_agent(
agent='chat-conversational-react-description',
tools=tools,
llm=model,
verbose=True,
max_iterations=10000,
early_stopping_method='generate',
memory=memory,
handle_parsing_errors=True)

c:\anaconda\envs\langgraph\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [10]:
conversational_agent("""In the given csv file change the format of birth-date column format into yyyy/mm//dd and dont change remaining columns and save it as people.csv file
                     """)
                    

c:\anaconda\envs\langgraph\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
```json
{
    "action": "Data_analyst",
    "action_input": "Please analyze the CSV file and change the format of the birth-date column to yyyy/mm/dd, leaving the remaining columns unchanged, and save it as people.csv."
}
```

c:\anaconda\envs\langgraph\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



Observation: The CSV file has been successfully analyzed, the 'birth-date' column has been converted to the desired format, and the resulting dataframe has been saved as people.csv.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The CSV file has been successfully analyzed, the 'birth-date' column has been converted to the desired format, and the resulting dataframe has been saved as people.csv."
}
```

> Finished chain.


{'input': 'In the given csv file change the format of birth-date column format into yyyy/mm//dd and dont change remaining columns and save it as people.csv file\n                     ',
 'chat_history': [],
 'output': "The CSV file has been successfully analyzed, the 'birth-date' column has been converted to the desired format, and the resulting dataframe has been saved as people.csv."}

In [12]:
conversational_agent("""In the given poeple.csv file create a new column called Name and fill the column with the column values of 
                     First Name and Last Name separated with space.Then Delete First Name and Last Name column in th dataset.
                     Save the given dataset after the modification as people.csv. 
                     """)
                    



> Entering new AgentExecutor chain...
```json
{
    "action": "Data_analyst",
    "action_input": "import pandas as pd; df = pd.read_csv('people.csv'); df['Name'] = df['First Name'] + ' ' + df['Last Name']; df = df.drop(['First Name', 'Last Name'], axis=1); df.to_csv('people.csv', index=False)"
}
```
Observation: The operations have been performed on the dataframe `df` and it has been saved to a CSV file named `people.csv`. The resulting CSV file will have the `Name` column created by concatenating `First Name` and `Last Name`, and the `First Name` and `Last Name` columns will be dropped.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The resulting dataframe has a new column called 'Name' which is a combination of 'First Name' and 'Last Name' separated by a space, and the 'First Name' and 'Last Name' columns have been deleted. The modified dataframe has been saved to a CSV file named 'people.csv'."
}
```

> Finished chain.


{'input': 'In the given poeple.csv file create a new column called Name and fill the column with the column values of \n                     First Name and Last Name separated with space.Then Delete First Name and Last Name column in th dataset.\n                     Save the given dataset after the modification as people.csv. \n                     ',
 'chat_history': [HumanMessage(content='In the given poeple.csv file create a new column called Name and fill the column with the column values of \n                     First Name and Last Name separated with space.Then Delete First Name and Last Name column in th dataset.\n                     Save the given dataset after the modification . \n                     '),
  AIMessage(content="The resulting dataframe has 8 columns: 'Index', 'User Id', 'Sex', 'Email', 'Phone', 'Job Title', 'birth-date', and 'Name'.")],
 'output': "The resulting dataframe has a new column called 'Name' which is a combination of 'First Name' and 'Last Name' sep

In [10]:
conversational_agent("""load every columns and rows from given csv file into the sql database(my_database.db).
                     Then using sql queries print the entries where the sex is male""")
                    



> Entering new AgentExecutor chain...


c:\anaconda\envs\langgraph\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


```json
{
    "action": "Data_analyst",
    "action_input": "Load the CSV file and import it into the SQL database. Then, use SQL queries to retrieve the entries where the sex is male."
}
```

c:\anaconda\envs\langgraph\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



Observation: The entries where the sex is male have been retrieved from the SQL database and printed out. There are 40 entries in total.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "There are 40 entries where the sex is male."
}
```

> Finished chain.


{'input': 'load every columns and rows from given csv file into the sql database.\n                     Then using sql queries print the entries where the sex is male',
 'chat_history': [],
 'output': 'There are 40 entries where the sex is male.'}